In [1]:
import os

# Get the directory containing the current notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# Change working directory to notebook folder
os.chdir(notebook_dir)

# Verify
print("Current working directory:", os.getcwd())

Current working directory: /home/j/jl1416/sta521/521PredictionProject-1


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder  
from xgboost import XGBClassifier
import optuna
from sklearn.metrics import cohen_kappa_score

# -----------------------------
# Load cleaned data
# -----------------------------
train = pd.read_csv('data/cleaned/train_cleaned.csv')
holdout = pd.read_csv('data/cleaned/test_cleaned.csv')

y_train = train['damage_grade']
X_train = train.drop(columns=['damage_grade'])

y_holdout = holdout['damage_grade']
X_holdout = holdout.drop(columns=['damage_grade'])

# Features to target‐encode
geo_target = ['geo__geo_level_2_id', 'geo__geo_level_3_id']

In [3]:
# -----------------------------
# Define Optuna objective
# -----------------------------
def objective(trial):
    # Hyperparameters for XGBoost
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 6),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 6),
        'eval_metric': 'mlogloss',
        'random_state': 42
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = []

    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr = X_train.iloc[train_idx].copy()
        X_val = X_train.iloc[val_idx].copy()
        y_tr = y_train.iloc[train_idx]
        y_val = y_train.iloc[val_idx]

        # Use sklearn TargetEncoder
        te = TargetEncoder(cv=5, shuffle=True, random_state=42, target_type = "multiclass")
        # Note: fit_transform does *cross‑fitting* internally in sklearn version.
        X_tr_enc = te.fit_transform(X_tr[geo_target], y_tr)
        X_val_enc = te.transform(X_val[geo_target])

        # Replace original geo columns with encoded ones
        X_tr_enc = pd.DataFrame(X_tr_enc, index=X_tr.index, columns=te.get_feature_names_out(geo_target))
        X_val_enc = pd.DataFrame(X_val_enc, index=X_val.index, columns=te.get_feature_names_out(geo_target))

        X_tr_full = X_tr.copy()
        X_val_full = X_val.copy()
        X_tr_full.drop(columns=geo_target, inplace=True)
        X_val_full.drop(columns=geo_target, inplace=True)
        # concat encoded
        X_tr_full = pd.concat([X_tr_full, X_tr_enc], axis=1)
        X_val_full = pd.concat([X_val_full, X_val_enc], axis=1)

        # Train model
        model = XGBClassifier(**params,
                              n_jobs=1)
        model.fit(X_tr_full, y_tr)
        
        y_pred = model.predict(X_val_full)

        acc = cohen_kappa_score(y_pred, y_val, weights = "quadratic")
        cv_scores.append(acc)

    # Return mean QWK so Optuna maximizes it
    return np.mean(cv_scores)

# -----------------------------
# Run Optuna
# -----------------------------
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, show_progress_bar=True, n_jobs=1)

print("Best params:", study.best_params)
print("Best CV QWK:", study.best_value)

[I 2025-11-22 20:19:13,663] A new study created in memory with name: no-name-ec21b678-50d2-4bdb-90af-c382097dd2d8


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2025-11-22 20:22:01,421] Trial 0 finished with value: 0.6051446621144072 and parameters: {'n_estimators': 926, 'max_depth': 8, 'learning_rate': 0.22637669365086402, 'subsample': 0.9949255698289003, 'colsample_bytree': 0.7453067441153655, 'gamma': 2.4898892957525005, 'reg_alpha': 1.7229498310313534, 'reg_lambda': 4.248822466434339}. Best is trial 0 with value: 0.6051446621144072.
[I 2025-11-22 20:25:32,673] Trial 1 finished with value: 0.6033029887294384 and parameters: {'n_estimators': 750, 'max_depth': 6, 'learning_rate': 0.024864075016412898, 'subsample': 0.7353384439787194, 'colsample_bytree': 0.8633114556937853, 'gamma': 3.6013436074647176, 'reg_alpha': 2.8406978591867613, 'reg_lambda': 1.6389753007637657}. Best is trial 0 with value: 0.6051446621144072.
[I 2025-11-22 20:28:33,627] Trial 2 finished with value: 0.602206988819284 and parameters: {'n_estimators': 1009, 'max_depth': 6, 'learning_rate': 0.25137804574381756, 'subsample': 0.7910566705288797, 'colsample_bytree': 0.54367

In [4]:
# -----------------------------
# Train final model on full train set with target encoding
# -----------------------------
te_final = TargetEncoder(cv=5, shuffle=True, random_state=42)
X_train_enc = te_final.fit_transform(X_train[geo_target], y_train)
X_holdout_enc = te_final.transform(X_holdout[geo_target])

X_train_enc = pd.DataFrame(X_train_enc, index=X_train.index, columns=te_final.get_feature_names_out(geo_target))
X_holdout_enc = pd.DataFrame(X_holdout_enc, index=X_holdout.index, columns=te_final.get_feature_names_out(geo_target))

X_train_full = X_train.copy()
X_holdout_full = X_holdout.copy()
X_train_full.drop(columns=geo_target, inplace=True)
X_holdout_full.drop(columns=geo_target, inplace=True)

X_train_full = pd.concat([X_train_full, X_train_enc], axis=1)
X_holdout_full = pd.concat([X_holdout_full, X_holdout_enc], axis=1)

final_model = XGBClassifier(**study.best_params, eval_metric='mlogloss', random_state=42, n_jobs=1)
final_model.fit(X_train_full, y_train)

# -----------------------------
# Evaluate on holdout
# -----------------------------
y_pred = final_model.predict(X_holdout_full)
holdout_acc = cohen_kappa_score(y_pred, y_holdout, weights = "quadratic")
print("Holdout QWK:", holdout_acc)


Holdout QWK: 0.6239224748119776


In [5]:
import pickle

with open("artifacts/final_model_xgb.pkl", "wb") as f:
    pickle.dump(final_model, f)